In [93]:
import datetime
from dateutil.parser import parse
import requests

class getData:
    
    def __init__(self):
        self.apiUrl = "https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/"
        
    def getSubjectList(self,study_id,domain_id):
        
        try:
            url = study_id + "/" + domain_id + "/" + "subject/list"
            URL = self.apiUrl + url
            
            req = requests.get(url = URL) 
            
            data = req.json()
            
            return data
        
        except e:
            print(e.message)
            return -1
    
    def getSubjectIdList(self,study_id,domain_id,subject_id):
        
        try:
            url = study_id + "/" + domain_id + "/subject/" + subject_id + "/list" 
            URL = self.apiUrl + url
            
            req = requests.get(url = URL) 
            
            data = req.json()
            
            return data
        
        except e:
            print(e.message)
            return -1
        
class identifyType():
    
    def __init__(self):
        self.type1 = "TYPE1"
        self.type2 = "TYPE2"
        self.type3 = "TYPE3"
        self.type4 = "TYPE4"
        self.type5 = "TYPE5"
    
    def identifyType(self,aeDataset,ceDataset):
        
        outputTypes = {}
        
        processedAeKeys = {}
        
        for i in range(len(aeDataset)):
            
            subjid = aeDataset[i]["subjid"]
            formname = aeDataset[i]["formname"]
            formid = aeDataset[i]["formid"]
            formidx = aeDataset[i]["formidx"]
            aeterm = aeDataset[i]["aeterm"]
            siteid = aeDataset[i]["siteid"]
                
            # checking if the type 3 error exists
            if (subjid,aeterm) in processedAeKeys:
                
                if (formname,formid,formidx,aeterm) in outputTypes and \
                self.type3 not in outputTypes[(formname,formid,formidx)]:
                    outputTypes[(formname,formid,formidx)].append(self.type3)        
                else:
                    outputTypes[(formname,formid,formidx)] = [self.type3]
                
            else:
                
                processedAeKeys[(subjid,aeterm)] = 1
            
                aestdat = aeDataset[i]["aestdat"]
                aeendat = aeDataset[i]["aeendat"]
            
                difference_in_days = 0
            
                if self.is_date(aestdat) and self.is_date(aeendat):
                    aestdat_dt = datetime.datetime.strptime(aestdat,'%d-%b-%y')
                    aeendat_dt = datetime.datetime.strptime(aeendat,'%d-%b-%y') 
                    difference_in_days = abs((aeendat_dt - aestdat_dt).days)
                
                if len(ceDataset) > 0:
                
                    ceOut = self.getceData(ceDataset,subjid,siteid)
                
                    if ceOut is not None:
                    
                        cmStartDt = ceOut[0]
                        cmEndDt = ceOut[1]
                        cmaeNo  = ceOut[2]
                    
                        # checking for type 1 error when aestdt is greater than medication end date 
                        if self.is_date(cmEndDt) and self.is_date(aestdat):
                            if cmEndDt < aestdat:
                                if (formname,formid,formidx,subjid) not in outputTypes:
                                    outputTypes[(formname,formid,formidx,subjid)] = [self.type1]
                                else:
                                    outputTypes[(formname,formid,formidx,subjid)].append(self.type1)
                                
                        # checking for type 2 error if difference in days is not equal to cmaeNo 
                        if len(cmaeNo) > 0 and difference_in_days != 0:
                            if difference_in_days != int(cmaeNo):
                                if (formname,formid,formidx,subjid) not in outputTypes:
                                    outputTypes[(formname,formid,formidx,subjid)] = [self.type2]
                                else:
                                    outputTypes[(formname,formid,formidx,subjid)].append(self.type2)
                                
         
        # checking for type 4 error
        
        processedCeKeys = {}
        
        for i in range(len(ceDataset)):
            
            subjid = ceDataset[i]["subjid"]
            formname = ceDataset[i]["formname"]
            formid = ceDataset[i]["formid"]
            formidx = ceDataset[i]["formidx"]
            cmtrt = ceDataset[i]["cmtrt"]      
            
            if (subjid,cmtrt) in processedCeKeys:
                
                if (formname,formid,formidx,aeterm) in outputTypes and \
                self.type4 not in outputTypes[(formname,formid,formidx,subjid)]:
                    outputTypes[(formname,formid,formidx,subjid)].append(self.type4)        
                else:
                    outputTypes[(formname,formid,formidx,subjid)] = [self.type4]  
            else:
                processedCeKeys[(subjid,cmtrt)] = 1
                            
        return outputTypes                

    def is_date(self,string, fuzzy=False):
        try: 
            parse(string, fuzzy=fuzzy)
            return True
        except ValueError:
            return False
    
    def getceData(self,ceDataset,subjid,siteid):
        
        for i in range(len(ceDataset)):
            if ceDataset[i]["subjid"] == subjid and ceDataset[i]["siteid"] == siteid:
                return (ceDataset[i]["cmstdat"],ceDataset[i]["cmendat"],ceDataset[i]["cmaeno"])
            
        return None     

In [ ]:
def main():
    
    objgetData = getData()
    objidentifyType = identifyType()
    
    dictSubjList = objgetData.getSubjectList("StudyHack","ae")
    
    postURL = "https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/api/1/{study_id}/query"
    
    if dictSubjList != -1:
        subjIdList = dictSubjList["data"]
        
        for i in range(len(subjIdList)):
            
            aeDataDict = objgetData.getSubjectIdList("StudyHack","ae",str(subjIdList[i]))
            ceDataDict = objgetData.getSubjectIdList("StudyHack","cm",str(subjIdList[i]))
            
            result = objidentifyType.identifyType(aeDataDict["data"],ceDataDict["data"])
            
            subjId = subjIdList[i]
            
            for key in result.keys():
                
                if len(key) == 4:
                    formname,formid,formidx,_ = key
                
                    for i in range(len(result[key])):
                    
                        PARAMS = {"email_address": "bsathyamur@gmail.com",
                                  "formname": formname,
                                  "formid": formid,
                                  "formidx": formidx,
                                  "type": result[key][i],
                                  "subjectid":subjId
                                 }
                    
                        resp = requests.get(url = URL,params = PARAMS) 
                    
                        print((formname,formid,formidx,subjId),resp)
                    
if __name__ == "__main__":
    main()

('AE202', 38768, 4, 76809) <Response [200]>
('AE202', 38768, 4, 76809) <Response [200]>
('AE202', 38768, 6, 76809) <Response [200]>
('AE202', 38768, 6, 76809) <Response [200]>
('AE202', 38768, 7, 76809) <Response [200]>
('AE202', 38768, 7, 76809) <Response [200]>
('AE202', 38768, 9, 76809) <Response [200]>
('AE202', 38768, 9, 76809) <Response [200]>
('AE202', 38768, 10, 76809) <Response [200]>
('AE202', 38768, 10, 76809) <Response [200]>
('AE202', 38768, 11, 76809) <Response [200]>
('AE202', 38768, 11, 76809) <Response [200]>
('AE202', 38768, 13, 76809) <Response [200]>
('AE202', 38768, 14, 76809) <Response [200]>
('AE202', 38768, 15, 76809) <Response [200]>
('AE202', 38768, 16, 76809) <Response [200]>
('AE202', 38768, 17, 76809) <Response [200]>
('AE202', 38768, 18, 76809) <Response [200]>
('AE202', 38768, 18, 76809) <Response [200]>
('AE202', 38768, 19, 76809) <Response [200]>
('AE202', 38768, 20, 76809) <Response [200]>
('AE202', 38768, 20, 76809) <Response [200]>
('AE202', 38768, 2